In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.fftpack import idct

%matplotlib inline
%config InlineBackend.figure_format='retina'

Variable initialization


In [ ]:
M = 32  # signal dimension
N = 2 * M  # number of atoms in the expansion


C = np.zeros(
    (M, M)
)  # matrix containing the standard basis (a kronecker delta in each column)
DCT = np.zeros(
    (M, M)
)  # matrix containing the DCT basis (a DCT function in each column)

Generate the 1D-DCT basis


In [ ]:
for i in range(M):
    DCT[:, i] = idct(np.eye(M)[:, i], type=2, norm="ortho")  # DCT basis

Generating the 1-D standard basis


In [ ]:
for i in range(M):
    C[:, i] = np.eye(M)[:, i]

Define the dictionary $D = [DCT, C]$


In [ ]:
D = np.hstack((DCT, C))

plt.figure(figsize=(10, 10))
plt.imshow(D)
plt.title(f"Our dictionary M = {M}, N = {N}")

## Generate a signal that is sparse w.r.t. $[DCT, C]$

To this purpose add a spike to the sum of few DCT atoms, i.e., add a spike to $\mathbf{s}$ that is sparse w.r.t. C. Bear in mind that the spike is to be considered a signal to be reconstructed, rather than noise


In [ ]:
L = 5
sigma_noise = 0.2

Randomly define the coefficients of a sparse representation w.r.t. $DCT$ (make sure the nonzero coefficients are sufficiently large)


In [ ]:
x0 = np.zeros(N)
nonzero_idx = np.random.choice(
    M, L, replace=False
)  # choose L unique indices in DCT part
x0[nonzero_idx] = np.random.randn(L) * 2 + np.random.choice([-1, 1], L) * 3

Choose spike location and update $x_0$


In [ ]:
spikeLocation = np.random.randint(M, N)
x0[spikeLocation] += -10

Synthesis the corresponding signal in the signal domain and add noise


In [ ]:
s0 = D @ x0
s = s0 + sigma_noise * np.random.normal(scale=2, size=M)

Plot the sparse signal


In [ ]:
LN_WDT = 2
MRK_SZ = 10

plt.figure(figsize=(6, 6))
plt.plot(s0, "b--", linewidth=LN_WDT + 1)
plt.plot(s, "r--x", linewidth=LN_WDT - 1)
plt.title(f"Sparse signal in DCT domain (L = {L:.0f})")
plt.legend(["original", "noisy"])

## Matching Pursuit

Initialize all the variables, including the residual, namely the components of the signals that can not be represented (here the signal at the very beginning)


In [ ]:
x_MP = np.zeros(N)

r = s.copy()  # initialize residual as the noisy signal (components not yet represented)

l = 1

# initialize the norm of the residual (components not represented by the coefficients)

resNorm = np.linalg.norm(r)  # L2 norm of the residual

MINIMUM_RES_NORM = 0.1

MP loop starts.

Stoppint criteria: continue until the sparsity of the representation reaches L or as long as resNorm(l) is above a minimum value or as long as a maxium number of iterations have been reached


In [ ]:
while np.count_nonzero(x_MP) < L and resNorm > MINIMUM_RES_NORM and l < 2 * L:
    # SWEEP STEP: look for the column of D that matches at best noisySignal
    # compute the residual w.r.t. each column of D
    e = np.zeros(N)
    for j in range(N):
        # Compute projection coefficient
        proj_coeff = np.dot(D[:, j], r)
        # Compute residual error after projection onto atom j
        e[j] = np.linalg.norm(r - proj_coeff * D[:, j]) ** 2
        # this corresponds to solving e(j) = min( || dj zj - r ||),
        # which is obtained by setting zj = dj' r / || dj ||^2 (analytically defined)
        # there is no need to divide by || dj ||^2 since columns are normalized

    # find the column of D that matches at best r, i.e. jStar = argmin(e(j))
    jStar = np.argmin(e)

    # UPDATE the jStar coefficient by *summing* the new component dj' r^(i) / || dj ||^2
    x_MP[jStar] += np.dot(D[:, jStar], r)

    # remove the signal we have so far represented in coeff_MP (update the residual)
    r = (
        r - np.dot(D[:, jStar], r) * D[:, jStar]
    )  # component that cannot be captured by the signal

    l = l + 1

    # update the residual norm
    resNorm = np.linalg.norm(r)

SYNTHESIS: reconstruct the signal, by inverting the transformation to reconstruct the signal


In [ ]:
s_hat_MP = D @ x_MP

Those part of the signal that have not been modeled by s_hat (i.e. the projection on the subspace of the L most involved coefficients) corresponds to the norm of the residual


In [ ]:
resNorm_MP = np.linalg.norm(s - s_hat_MP)

Show the result


In [ ]:
LN_WDT = 2
MRK_SZ = 10

fix, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(s0, "b-o", linewidth=LN_WDT + 1)
ax[0].plot(s, "r-x", linewidth=LN_WDT - 1)
ax[0].plot(s_hat_MP, "m-", linewidth=LN_WDT)
ax[0].set_title(f"Sparse signal in DCT domain (L = {L:.0f})")
ax[0].legend(["original (s0)", "noisy (s)", "MP estimate"])

ax[1].stem(x0, linefmt="b-", markerfmt="C0o")
ax[1].stem(x_MP, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("DCT Coefficients")
ax[1].legend(["coefficients of s0 (x0)", "coefficients of s_hat (x_hat)"])